In [16]:
import polars as pl
import pytz
import yfinance as yf
from datetime import datetime

In [2]:
df_tqqq = pl.from_pandas( yf.ticker.Ticker("TQQQ").history(interval="1h", start="2025-02-26", end="2025-03-26").reset_index(names="ts_event") )
df_tqqq = df_tqqq.with_columns(
    (pl.col("ts_event") - pl.duration(minutes=30)).cast(pl.Datetime("ns", time_zone="America/New_York"))
)
df_tqqq

ts_event,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
"datetime[ns, America/New_York]",f64,f64,f64,f64,i64,f64,f64,f64
2025-02-26 09:00:00 EST,78.57,80.050003,78.047997,79.065002,17641275,0.0,0.0,0.0
2025-02-26 10:00:00 EST,79.059998,80.400002,79.059998,79.974998,7941396,0.0,0.0,0.0
2025-02-26 11:00:00 EST,79.970001,80.449898,79.629997,80.110001,6072975,0.0,0.0,0.0
2025-02-26 12:00:00 EST,80.139999,80.300003,77.845001,77.860001,9273837,0.0,0.0,0.0
2025-02-26 13:00:00 EST,77.82,78.220001,76.690002,77.3731,8811807,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…
2025-03-25 11:00:00 EDT,67.184998,67.203697,66.720001,66.764999,5822048,0.0,0.0,0.0
2025-03-25 12:00:00 EDT,66.769897,67.230003,66.129997,66.535004,5652755,0.0,0.0,0.0
2025-03-25 13:00:00 EDT,66.540001,66.995003,66.4701,66.775002,4708939,0.0,0.0,0.0


In [3]:
options_hourly_data_file = r"C:\Users\User\Desktop\projects\trading\data\OPRA-20250329-XW7CFKR3PW\opra-pillar-20250226-20250325.ohlcv-1h.csv"

In [ ]:
from scipy.stats import norm
from scipy.optimize import brentq
import numpy as np

# Black-Scholes price for PUT
def bs_put_price(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r*T) * norm.cdf(-d2) - S * norm.cdf(-d1)

# Black-Scholes price for CALL
def bs_call_price(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)

# Scalar root-finder: implied vol for PUT
def implied_vol_put_scalar(S, K, T, r, P_market):
    if P_market <= 0 or S <= 0 or K <= 0 or T <= 0:
        return np.nan
    try:
        return brentq(lambda sigma: bs_put_price(S, K, T, r, sigma) - P_market, 1e-6, 5.0)
    except:
        return np.nan

# Scalar root-finder: implied vol for CALL
def implied_vol_call_scalar(S, K, T, r, C_market):
    if C_market <= 0 or S <= 0 or K <= 0 or T <= 0:
        return np.nan
    try:
        return brentq(lambda sigma: bs_call_price(S, K, T, r, sigma) - C_market, 1e-6, 5.0)
    except:
        return np.nan


In [27]:
pl.read_csv(options_hourly_data_file) \
    .with_columns(
        pl.col("symbol").str.split("  ")
    ) \
    .with_columns(
        pl.col("ts_event").str.strptime( pl.Datetime("ns", time_zone=pytz.timezone("America/New_York")) ),
        pl.col("symbol").list.get(0).alias("underlying"),
        pl.col("symbol").list.get(-1).str.slice(0, 6).str.strptime(pl.Date, "%y%m%d").cast(pl.Datetime("ns")).dt.replace_time_zone("America/New_York").alias("expiry"),
        pl.col("symbol").list.get(-1).str.slice(6, 1).alias("call_put"),
        (pl.col("symbol").list.get(-1).str.slice(7).cast(pl.Int64) / 1000.0).alias("strike")
    ) \
    .with_columns(
        (pl.col("expiry") - pl.col("ts_event")).dt.total_days().alias("days_to_expiry"),
    ) \
    .join( df_tqqq, on="ts_event", how="left") \
    .filter( pl.col("ts_event") < pl.lit("2025-02-27").str.strptime(pl.Datetime("ns", time_zone="America/New_York") ) ) \
    .with_columns(
        pl.struct(["Close", "strike", "days_to_expiry", "close", "call_put"]).map_elements(
            lambda row: implied_vol_put_scalar(
                S=row["Close"],
                K=row["strike"],
                T=row["days_to_expiry"] / 365,
                r=0.05,
                P_market=row["close"]
            ) if row["call_put"] == "P" else np.nan,
            return_dtype=pl.Float32
        ).alias("implied_vol")
    )

ts_event,rtype,publisher_id,instrument_id,open,high,low,close,volume,symbol,underlying,expiry,call_put,strike,days_to_expiry,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,implied_vol
"datetime[ns, America/New_York]",i64,i64,i64,f64,f64,f64,f64,i64,list[str],str,"datetime[ns, America/New_York]",str,f64,i64,f64,f64,f64,f64,i64,f64,f64,f64,f32
2025-02-26 09:00:00 EST,34,32,1426078889,5.4,5.4,5.4,5.4,4,"[""TQQQ"", ""250321P00080000""]","""TQQQ""",2025-03-21 00:00:00 EDT,"""P""",80.0,22,78.57,80.050003,78.047997,79.065002,17641275,0.0,0.0,0.0,0.649359
2025-02-26 09:00:00 EST,34,29,1426078889,5.4,5.4,5.4,5.4,7,"[""TQQQ"", ""250321P00080000""]","""TQQQ""",2025-03-21 00:00:00 EDT,"""P""",80.0,22,78.57,80.050003,78.047997,79.065002,17641275,0.0,0.0,0.0,0.649359
2025-02-26 09:00:00 EST,34,35,1442848551,4.0,4.0,4.0,4.0,1,"[""TQQQ"", ""250228P00081000""]","""TQQQ""",2025-02-28 00:00:00 EST,"""P""",81.0,1,78.57,80.050003,78.047997,79.065002,17641275,0.0,0.0,0.0,1.759063
2025-02-26 09:00:00 EST,34,32,1442865848,2.58,2.58,2.14,2.37,37,"[""TQQQ"", ""250228P00079000""]","""TQQQ""",2025-02-28 00:00:00 EST,"""P""",79.0,1,78.57,80.050003,78.047997,79.065002,17641275,0.0,0.0,0.0,1.459393
2025-02-26 09:00:00 EST,34,33,1442865848,2.37,2.37,2.37,2.37,1,"[""TQQQ"", ""250228P00079000""]","""TQQQ""",2025-02-28 00:00:00 EST,"""P""",79.0,1,78.57,80.050003,78.047997,79.065002,17641275,0.0,0.0,0.0,1.459393
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025-02-26 15:00:00 EST,34,26,1442866478,0.83,0.95,0.83,0.95,2,"[""TQQQ"", ""250328C00091000""]","""TQQQ""",2025-03-28 00:00:00 EDT,"""C""",91.0,29,77.610001,78.43,77.379997,78.07,4177007,0.0,0.0,0.0,NaN
2025-02-26 15:00:00 EST,34,22,1442842807,8.0,8.0,6.95,6.95,10,"[""TQQQ"", ""250228P00085000""]","""TQQQ""",2025-02-28 00:00:00 EST,"""P""",85.0,1,77.610001,78.43,77.379997,78.07,4177007,0.0,0.0,0.0,0.820786
2025-02-26 15:00:00 EST,34,28,1442842807,7.47,7.47,7.47,7.47,1,"[""TQQQ"", ""250228P00085000""]","""TQQQ""",2025-02-28 00:00:00 EST,"""P""",85.0,1,77.610001,78.43,77.379997,78.07,4177007,0.0,0.0,0.0,1.598461
